In [1]:
import os
import numpy as np
# import pandas as pd
import h5py
# from matplotlib import pyplot as plt
# %matplotlib inline
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import Normalizer
from sklearn.metrics import roc_auc_score
import keras #Change later once I have metrics done/
import my_callbacks
from keras.utils import np_utils
import my_metrics

Using TensorFlow backend.


In [2]:
# x = h5py.File('/eos/cms/store/user/fsiroky/hdf5_data/MET_C_background.h5', 'r')
# x = h5py.File('/afs/cern.ch/user/f/fsiroky/public/JetHC_Cbackground.h5', 'r')

In [3]:
# Where the .h5 data is stored
# b_h5 = '/eos/cms/store/user/fsiroky/hdf5_data/'
# b_h5 = '/mnt/hdf5test/'
b_h5   = '/home/test_local/'

pds  = {1: 'BTagCSV', 2: 'BTagMu', 3: 'Charmonium', 4:'DisplacedJet', 5: 'DoubleEG',
        6: 'DoubleMuon', 7: 'DoubleMuonLowMass',
       # 8: 'FSQJets', 9: 'HighMultiplicityEOF', #NOT ENOUGH DATA, NOTEBOOK FAILES
        10: 'HTMHT', 11: 'JetHT', 12: 'MET',
       # 13: 'MinimumBias', #NOT ENOUGH DATA
        14: 'MuonEG', 15: 'MuOnia',
       # 16: 'NoBPTX',
        17: 'SingleElectron', 18: 'SingleMuon', 19: 'SinglePhoton', 20: 'Tau', 21: 'ZeroBias'
}
nbr = 11

bg_files  = [b_h5+pds[nbr]+'_C_background.h5',b_h5+pds[nbr]+'_D_background.h5', b_h5+pds[nbr]+'_E_background.h5',
             b_h5+pds[nbr]+'_F_background.h5', b_h5+pds[nbr]+'_G_background.h5', b_h5+pds[nbr]+'_H_background.h5']

bg_jets   = [pds[nbr]+"_C_background", pds[nbr]+"_D_background", pds[nbr]+"_E_background",
             pds[nbr]+"_F_background", pds[nbr]+"_G_background", pds[nbr]+"_H_background"]

sig_files = [b_h5+pds[nbr]+'_C_signal.h5',b_h5+pds[nbr]+'_D_signal.h5', b_h5+pds[nbr]+'_E_signal.h5',
             b_h5+pds[nbr]+'_F_signal.h5', b_h5+pds[nbr]+'_G_signal.h5', b_h5+pds[nbr]+'_H_signal.h5']

sig_jets  = [pds[nbr]+"_C_signal", pds[nbr]+"_D_signal", pds[nbr]+"_E_signal",
             pds[nbr]+"_F_signal", pds[nbr]+"_G_signal", pds[nbr]+"_H_signal"]


def get_jets(bg_files, bg_jets, sig_files, sig_jets, ratio=1.0):
    good_jets = np.empty([0,2802])
    bad_jets  = np.empty([0,2802])
    np.random.seed(24)
    for i in range(len(bg_files)):
        try:
            bg_jetfile  = h5py.File(bg_files[i],'r')
            bg_jet      = bg_jetfile[bg_jets[i]][:]
            sig_jetfile = h5py.File(sig_files[i],'r')
            sig_jet     = sig_jetfile[sig_jets[i]][:]

            bad_jets    = np.concatenate((bad_jets, bg_jet), axis=0)
            try:
                ratio_abs   = ratio*len(bg_jet)/len(sig_jet)
                idx = np.arange(sig_jet.shape[0])
                mask = np.zeros_like(idx, dtype=bool)

                selected = np.random.choice(idx, len(bg_jet)*ratio, replace=False)
                mask[selected] = True

                good_jets = np.concatenate((good_jets, sig_jet[mask]), axis=0)
                print(np.sum(mask), len(bg_jet),  len(good_jets), ratio_abs) #FIX HERE
            except:
                print("Size of sig_jet of ", sig_jets[i], " is 0")
#                 ratio_abs = str("Size of sig_jet is " + len(sig_jet)) 

        except OSError:
            print("This Primary Dataset doesn't have ", bg_jets[i])
    return good_jets, bad_jets

      

def get_train_test_split(good_jets, bad_jets):
    x_train = np.concatenate((good_jets,bad_jets), axis=0)
    scaler = Normalizer()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)

    y_train = np.zeros(len(x_train))
    y_train[0:len(good_jets)]=1
    mask = shuffle((np.arange(len(x_train))), random_state=42)

    x_train = x_train[mask]
    y_train = y_train[mask]

    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [ ]:
%%time

ratios = [0.5,1,3,5]

BTagCSV_scores, BTagCSV_aucs, BTagCSV_goodbad                                     = [],[],[]
BTagMu_scores, BTagMu_aucs, BTagMu_goodbad                                        = [],[],[]
Charmonium_scores, Charmonium_aucs, Charmonium_goodbad                            = [],[],[] 
DisplacedJet_scores, DisplacedJet_aucs, DisplacedJet_goodbad                      = [],[],[]
DoubleEG_scores, DoubleEG_aucs, DoubleEG_goodbad                                  = [],[],[]
DoubleMuon_scores, DoubleMuon_aucs, DoubleMuon_goodbad                            = [],[],[]              
DoubleMuonLowMass_scores, DoubleMuonLowMass_aucs, DoubleMuonLowMass_goodbad       = [],[],[] 
FSQJets_scores, FSQJets_aucs, FSQJets_goodbad                                     = [],[],[]
HighMultiplicityEOF_scores, HighMultiplicityEOF_aucs, HighMultiplicityEOF_goodbad = [],[],[] 
HTMHT_scores, HTMHT_aucs, HTMHT_goodbad                                           = [],[],[]
JetHT_scores, JetHT_aucs, JetHT_goodbad                                           = [],[],[]
MET_scores, MET_aucs, MET_goodbad                                                 = [],[],[]
MinimumBias_scores, MinimumBias_aucs, MinimumBias_goodbad                         = [],[],[]
MuonEG_scores, MuonEG_aucs, MuonEG_goodbad                                        = [],[],[]
MuOnia_scores, MuOnia_aucs, MuOnia_goodbad                                        = [],[],[]
NoBPTX_scores, NoBPTX_aucs, NoBPTX_goodbad                                        = [],[],[]
SingleElectron_scores, SingleElectron_aucs, SingleElectron_goodbad                = [],[],[]
SingleMuon_scores, SingleMuon_aucs, SingleMuon_goodbad                            = [],[],[]
SinglePhoton_scores, SinglePhoton_aucs, SinglePhoton_goodbad                      = [],[],[]
Tau_scores, Tau_aucs, Tau_goodbad                                                 = [],[],[]
ZeroBias_scores, ZeroBias_aucs, ZeroBias_goodbad                                  = [],[],[]
           

for pd in range (1,22):
    goodbad_ratio = []
    scores   = []
    roc_a    = []
    try:
        print(pds[pd])
    except KeyError:
        print("PD with number ", pd, " was commented out")
        continue
    #Ratios meant in the subsampling sense    
    for rat in range (0,len(ratios)):
    
        nbr = pd
        print("Primary dataset read and subsampled with ratio ", ratios[rat], " from" , b_h5, "is ", pds[nbr])
        bg_files  = [b_h5+pds[nbr]+'_C_background.h5',b_h5+pds[nbr]+'_D_background.h5', b_h5+pds[nbr]+'_E_background.h5',
                 b_h5+pds[nbr]+'_F_background.h5', b_h5+pds[nbr]+'_G_background.h5', b_h5+pds[nbr]+'_H_background.h5']

        bg_jets   = [pds[nbr]+"_C_background", pds[nbr]+"_D_background", pds[nbr]+"_E_background",
                 pds[nbr]+"_F_background", pds[nbr]+"_G_background", pds[nbr]+"_H_background"]

        sig_files = [b_h5+pds[nbr]+'_C_signal.h5',b_h5+pds[nbr]+'_D_signal.h5', b_h5+pds[nbr]+'_E_signal.h5',
                 b_h5+pds[nbr]+'_F_signal.h5', b_h5+pds[nbr]+'_G_signal.h5', b_h5+pds[nbr]+'_H_signal.h5']

        sig_jets  = [pds[nbr]+"_C_signal", pds[nbr]+"_D_signal", pds[nbr]+"_E_signal",
                 pds[nbr]+"_F_signal", pds[nbr]+"_G_signal", pds[nbr]+"_H_signal"]
    
        good_jets, bad_jets = get_jets(bg_files, bg_jets, sig_files, sig_jets, ratio=ratios[rat])
        x_train, x_test, y_train, y_test = get_train_test_split(good_jets, bad_jets)
        try:
            goodbad_ratio.append((len(good_jets)/(len(bad_jets)+len(good_jets))))
        except:
            goodbad_ratio.append("Size of sig_jet is 0 ")
        # Second split for validation set
        del(good_jets)
        del(bad_jets)
        x_train, x_val, y_train, y_val   = train_test_split(x_train, y_train, test_size=0.15, random_state=1)

        model = Sequential()
        model.add(Dense(2000, input_dim=2802, activation='relu'))
        model.add(Dropout(0.1))
        model.add(Dense(1000, activation='relu'))
        model.add(Dropout(0.1))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy', my_metrics.f1, my_metrics.mcor])
        # Uncomment class_weight for setting higher weights for outliers.
#         class_weight = {0 : 1.,
#                         1 : 10.}
        model.fit(x_train, y_train, 
                  validation_data=(x_val,y_val),
                  epochs=20,
                  batch_size=32
#                   class_weight = class_weight
                  )

        score = model.evaluate(x_test, y_test, batch_size=32)
        y_pred = model.predict(x_test)
        print()
        roc_a.append(roc_auc_score(y_test, y_pred))
        print("ROC AUC SCORE for ratio", ratios[rat], "for PD", pds[pd], "is", roc_a)

        scores.append(score)
        acc = [e[1] for e in scores]
        print("ACCURACY for ratio", ratios[rat], "for PD", pds[pd], "is", acc)
#         goodbad_ratio.append((len(good_jets)/(len(bad_jets)+len(good_jets))))
        print("GOOD/BAD ratio for ratio", ratios[rat], "for PD", pds[pd], "is", goodbad_ratio)
        print("xxxxxxxxxx")
        del(x_train)
        del(x_test)
        del(x_val)
        
        #Uncomment for writing results in log text file
#         if pd == 1:
#             BTagCSV_scores.append(acc[rat]), BTagCSV_aucs.append(roc_a[rat]), BTagCSV_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('BTagCSV split ratio ' + str(ratios[rat]) + '\n' + 'BTagCSV_accuracy is ' +  str(BTagCSV_scores[rat]) +
#                          ' BTagCSV_auc is ' + str(BTagCSV_aucs[rat]) + ' BTagCSV_goodbad ratio is ' + str(BTagCSV_goodbad[rat]) + '\n')                  
#         if pd == 2:
#             BTagMu_scores.append(acc[rat]), BTagMu_aucs.append(roc_a[rat]), BTagMu_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('BTagMu split ratio ' + str(ratios[rat]) + '\n' + 'BTagMu_accuracy is ' +  str(BTagMu_scores[rat]) +
#                          ' BTagMu_auc is ' + str(BTagMu_aucs[rat]) + ' BTagMu_goodbad ratio is ' + str(BTagMu_goodbad[rat]) + '\n')   
#         if pd == 3:
#             Charmonium_scores.append(acc[rat]), Charmonium_aucs.append(roc_a[rat]), Charmonium_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('Charmonium split ratio ' + str(ratios[rat]) + '\n' + 'Charmonium_accuracy is ' +  str(Charmonium_scores[rat]) +
#                          ' Charmonium_auc is ' + str(Charmonium_aucs[rat]) + ' Charmonium_goodbad ratio is ' + str(Charmonium_goodbad[rat]) + '\n')   
#         if pd == 4:
#             DisplacedJet_scores.append(acc[rat]), DisplacedJet_aucs.append(roc_a[rat]), DisplacedJet_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('DisplacedJet split ratio ' + str(ratios[rat]) + '\n' + 'DisplacedJet_accuracy is ' +  str(DisplacedJet_scores[rat]) +
#                          ' DisplacedJet_auc is ' + str(DisplacedJet_aucs[rat]) + ' DisplacedJet_goodbad ratio is ' + str(DisplacedJet_goodbad[rat]) + '\n')   
#         if pd == 5:
#             DoubleEG_scores.append(acc[rat]), DoubleEG_aucs.append(roc_a[rat]), DoubleEG_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('DoubleEG split ratio ' + str(ratios[rat]) + '\n' + 'DoubleEG_accuracy is ' +  str(DoubleEG_scores[rat]) +
#                          ' DoubleEG_auc is ' + str(DoubleEG_aucs[rat]) + ' DoubleEG_goodbad ratio is ' + str(DoubleEG_goodbad[rat]) + '\n')   
#         if pd == 6:
#             DoubleMuon_scores.append(acc[rat]), DoubleMuon_aucs.append(roc_a[rat]), DoubleMuon_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('DoubleMuon split ratio ' + str(ratios[rat]) + '\n' + 'DoubleMuon_accuracy is ' +  str(DoubleMuon_scores[rat]) +
#                          ' DoubleMuon_auc is ' + str(DoubleMuon_aucs[rat]) + ' DoubleMuon_goodbad ratio is ' + str(DoubleMuon_goodbad[rat]) + '\n')   
#         if pd == 7:
#             DoubleMuonLowMass_scores.append(acc[rat]), DoubleMuonLowMass_aucs.append(roc_a[rat]), DoubleMuonLowMass_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('DoubleMuonLowMass split ratio ' + str(ratios[rat]) + '\n' + 'DoubleMuonLowMass_accuracy is ' +  str(DoubleMuonLowMass_scores[rat]) +
#                          ' DoubleMuonLowMass_auc is ' + str(DoubleMuonLowMass_aucs[rat]) + ' DoubleMuonLowMass_goodbad ratio is ' + str(DoubleMuonLowMass_goodbad[rat]) + '\n')   
#         if pd == 8:
#             FSQJets_scores.append(acc[rat]), FSQJets_aucs.append(roc_a[rat]), FSQJets_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('FSQJets split ratio ' + str(ratios[rat]) + '\n' + 'FSQJets_accuracy is ' +  str(FSQJets_scores[rat]) +
#                          ' FSQJets_auc is ' + str(FSQJets_aucs[rat]) + ' FSQJets_goodbad ratio is ' + str(FSQJets_goodbad[rat]) + '\n')   
#         if pd == 9:
#             HighMultiplicityEOF_scores.append(acc[rat]), HighMultiplicityEOF_aucs.append(roc_a[rat]), HighMultiplicityEOF_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('HighMultiplicityEOF split ratio ' + str(ratios[rat]) + '\n' + 'HighMultiplicityEOF_accuracy is ' +  str(HighMultiplicityEOF_scores[rat]) +
#                          ' HighMultiplicityEOF_auc is ' + str(HighMultiplicityEOF_aucs[rat]) + ' HighMultiplicityEOF_goodbad ratio is ' + str(HighMultiplicityEOF_goodbad[rat]) + '\n')   
#         if pd == 10:
#             HTMHT_scores.append(acc[rat]), HTMHT_aucs.append(roc_a[rat]), HTMHT_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('HTMHT split ratio ' + str(ratios[rat]) + '\n' + 'HTMHT_accuracy is ' +  str(HTMHT_scores[rat]) +
#                          ' HTMHT_auc is ' + str(HTMHT_aucs[rat]) + ' HTMHT_goodbad ratio is ' + str(HTMHT_goodbad[rat]) + '\n')   
#         if pd == 11:
#             JetHT_scores.append(acc[rat]), JetHT_aucs.append(roc_a[rat]), JetHT_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('JetHT split ratio ' + str(ratios[rat]) + '\n' + 'JetHT_accuracy is ' +  str(JetHT_scores[rat]) +
#                          ' JetHT_auc is ' + str(JetHT_aucs[rat]) + ' JetHT_goodbad ratio is ' + str(JetHT_goodbad[rat]) + '\n')   
#         if pd == 12:
#             MET_scores.append(acc[rat]), MET_aucs.append(roc_a[rat]), MET_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('MET split ratio ' + str(ratios[rat]) + '\n' + 'MET_accuracy is ' +  str(MET_scores[rat]) +
#                          ' MET_auc is ' + str(MET_aucs[rat]) + ' MET_goodbad ratio is ' + str(MET_goodbad[rat]) + '\n')   
#         if pd == 13:
#             MinimumBias_scores.append(acc[rat]), MinimumBias_aucs.append(roc_a[rat]), MinimumBias_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('MinimumBias split ratio ' + str(ratios[rat]) + '\n' + 'MinimumBias_accuracy is ' +  str(MinimumBias_scores[rat]) +
#                          ' MinimumBias_auc is ' + str(MinimumBias_aucs[rat]) + ' MinimumBias_goodbad ratio is ' + str(MinimumBias_goodbad[rat]) + '\n')   
#         if pd == 14:
#             MuonEG_scores.append(acc[rat]), MuonEG_aucs.append(roc_a[rat]), MuonEG_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('MuonEG split ratio ' + str(ratios[rat]) + '\n' + 'MuonEG_accuracy is ' +  str(MuonEG_scores[rat]) +
#                          ' MuonEG_auc is ' + str(MuonEG_aucs[rat]) + ' MuonEG_goodbad ratio is ' + str(MuonEG_goodbad[rat]) + '\n')   
#         if pd == 15:
#             MuOnia_scores.append(acc[rat]), MuOnia_aucs.append(roc_a[rat]), MuOnia_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('MuOnia split ratio ' + str(ratios[rat]) + '\n' + 'MuOnia_accuracy is ' +  str(MuOnia_scores[rat]) +
#                          ' MuOnia_auc is ' + str(MuOnia_aucs[rat]) + ' MuOnia_goodbad ratio is ' + str(MuOnia_goodbad[rat]) + '\n')   
#         if pd == 16:
#             NoBPTX_scores.append(acc[rat]), NoBPTX_aucs.append(roc_a[rat]), NoBPTX_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('NoBPTX split ratio ' + str(ratios[rat]) + '\n' + 'NoBPTX_accuracy is ' +  str(NoBPTX_scores[rat]) +
#                          ' NoBPTX_auc is ' + str(NoBPTX_aucs[rat]) + ' NoBPTX_goodbad ratio is ' + str(NoBPTX_goodbad[rat]) + '\n')   
#         if pd == 17:
#             SingleElectron_scores.append(acc[rat]), SingleElectron_aucs.append(roc_a[rat]), SingleElectron_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('SingleElectron split ratio ' + str(ratios[rat]) + '\n' + 'SingleElectron_accuracy is ' +  str(SingleElectron_scores[rat]) +
#                          ' SingleElectron_auc is ' + str(SingleElectron_aucs[rat]) + ' SingleElectron_goodbad ratio is ' + str(SingleElectron_goodbad[rat]) + '\n')   
#         if pd == 18:
#             SingleMuon_scores.append(acc[rat]), SingleMuon_aucs.append(roc_a[rat]), SingleMuon_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('SingleMuon split ratio ' + str(ratios[rat]) + '\n' + 'SingleMuon_accuracy is ' +  str(SingleMuon_scores[rat]) +
#                          ' SingleMuon_auc is ' + str(SingleMuon_aucs[rat]) + ' SingleMuon_goodbad ratio is ' + str(SingleMuon_goodbad[rat]) + '\n')   
#         if pd == 19:
#             SinglePhoton_scores.append(acc[rat]), SinglePhoton_aucs.append(roc_a[rat]), SinglePhoton_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('SinglePhoton split ratio ' + str(ratios[rat]) + '\n' + 'SinglePhoton_accuracy is ' +  str(SinglePhoton_scores[rat]) +
#                          ' SinglePhoton_auc is ' + str(SinglePhoton_aucs[rat]) + ' SinglePhoton_goodbad ratio is ' + str(SinglePhoton_goodbad[rat]) + '\n')   
#         if pd == 20:
#             Tau_scores.append(acc[rat]), Tau_aucs.append(roc_a[rat]), Tau_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('Tau split ratio ' + str(ratios[rat]) + '\n' + 'Tau_accuracy is ' +  str(Tau_scores[rat]) +
#                          ' Tau_auc is ' + str(Tau_aucs[rat]) + ' Tau_goodbad ratio is ' + str(Tau_goodbad[rat]) + '\n')   
#         if pd == 21:
#             ZeroBias_scores.append(acc[rat]), ZeroBias_aucs.append(roc_a[rat]), ZeroBias_goodbad.append(goodbad_ratio[rat])
#             with open("test_log.txt", "a") as myfile:
#                 myfile.write('ZeroBias split ratio ' + str(ratios[rat]) + '\n' + 'ZeroBias_accuracy is ' +  str(ZeroBias_scores[rat]) +
#                          ' ZeroBias_auc is ' + str(ZeroBias_aucs[rat]) + ' ZeroBias_goodbad ratio is ' + str(ZeroBias_goodbad[rat]) + '\n')   

            

In [ ]:
# %%time

# scores = []
# ratios = [1]
# xy_ratio=[]

# for e in ratios:
#     good_jets, bad_jets = get_jets(bg_files, bg_jets, sig_files, sig_jets, ratio=e)
#     x_train, x_test, y_train, y_test = get_train_test_split(good_jets, bad_jets)
#     x_train, x_val, y_train, y_val   = train_test_split(x_train, y_train, test_size=0.15, random_state=1)

#     model = Sequential()
#     model.add(Dense(2802, input_dim=2802, activation='relu'))
#     model.add(Dropout(0.1))
#     model.add(Dense(1000, activation='relu'))
#     model.add(Dropout(0.1))
#     model.add(Dense(1, activation='sigmoid'))

#     model.compile(loss='binary_crossentropy',
#                   optimizer='Adam',
#                   metrics=['accuracy',my_metrics.f1,my_metrics.mcor])
# #     callbacks=[roc_callback(x_train=x_train,y_train=y_train,x_test=x_test,y_test=y_test)]
# #     histories = my_callbacks.Histories()
# #     class_weight = {0 : 1.,
# #     1: 10.}
#     model.fit(x_train, y_train, 
# #               validation_split=0.15,
#               validation_data=(x_val,y_val),
#               epochs=40,
#               batch_size=256)
# #              class_weight = class_weight)
# #              callbacks=[histories],
# #              validation_data=(x_test, y_test))
# #     print("callbacks ",callbacks[0].on_epoch_end(0))
# #     print("Losses are ",histories.losses)
# #     print("Aucs are ",histories.aucs)

#     score = model.evaluate(x_test, y_test, batch_size=256)
#     y_pred = model.predict(x_test)
#     print()
#     print("ROC AUC SCORE is",roc_auc_score(y_test, y_pred))



#     scores.append(score)
#     acc = [e[1] for e in scores]
#     print()
#     print(acc)
#     xy_ratio.append((len(good_jets)/(len(bad_jets)+len(good_jets))))
#     print(xy_ratio)
#     print("xxxxxxxxxx")

In [ ]:
whos